In [1]:
# Load model directly
from modelscope import Qwen2_5_VLForConditionalGeneration, Qwen2_5_VLProcessor

processor = Qwen2_5_VLProcessor.from_pretrained("Qwen-2.5-VL", trust_remote_code=True)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained("Qwen-2.5-VL", trust_remote_code=True)

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.76s/it]


In [2]:
conversation = [
    {
        "role": "system",
        "content": "You are a helpful AI assistant that can understand and describe videos. Please provide clear and detailed descriptions of video content when asked."
    },
    
]
text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=False)
print(text_prompt)

<|im_start|>system
You are a helpful AI assistant that can understand and describe videos. Please provide clear and detailed descriptions of video content when asked.<|im_end|>



In [3]:
conversation = [

    {
        "role": "user",
        "content": [
            {"type": "video", "video": "/scratch2/tannicholas/data/videos/1.mp4"},
            {"type": "text", "text": "What happened in the video?"},
        ],
    },
    
]
text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=False)
print(text_prompt)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<|vision_start|><|video_pad|><|vision_end|>What happened in the video?<|im_end|>



In [4]:
conversation = [
    {
        "role": "system",
        "content": "You are a helpful AI assistant that can understand and describe videos. Please provide clear and detailed descriptions of video content when asked."
    },
    {
        "role": "user",
        "content": [
            {"type": "video", "video": "/scratch2/tannicholas/data/videos/1.mp4"},
            {"type": "text", "text": "What happened in the video?"},
        ],
    },
    {
        "role": "assistant",
        "content": "I'll describe what I observed in the video. Please share the video and I'll provide a detailed description."
    }
]
text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=False)
print(text_prompt)

<|im_start|>system
You are a helpful AI assistant that can understand and describe videos. Please provide clear and detailed descriptions of video content when asked.<|im_end|>
<|im_start|>user
<|vision_start|><|video_pad|><|vision_end|>What happened in the video?<|im_end|>
<|im_start|>assistant
I'll describe what I observed in the video. Please share the video and I'll provide a detailed description.<|im_end|>



In [5]:

import torch



input_tensor = torch.randn(2, 3)  # A random tensor of size 2x3

filled_tensor = torch.full_like(input_tensor, 0)  # Creates a tensor with the same size as input_tensor, filled with 0s



print(filled_tensor) 


tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [6]:
from qwen_vl_utils import process_vision_info
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "../PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/train/01April_2010_Thursday_heute-6694.mp4",
                "fps": 25
            },
            {"type": "text", "text": "Describe this video."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
print(video_inputs[0].shape)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

qwen-vl-utils using decord to read video.


torch.Size([52, 3, 364, 308])


In [7]:
print(inputs.keys())

dict_keys(['input_ids', 'attention_mask', 'pixel_values_videos', 'video_grid_thw', 'second_per_grid_ts'])


In [8]:
inputs['attention_mask']

tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')

In [9]:
# default: Load the model on the available device(s)
import torch
from transformers import Qwen2_5_VLForConditionalGeneration
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen-2.5-VL", torch_dtype="auto", device_map="auto"
)
model.eval()
with torch.no_grad():
    generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]   
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.79s/it]


['A woman with shoulder-length brown hair is wearing a black blazer and a black shirt. She is standing in front of a plain background. She is making hand gestures, moving her hands up and down and in front of her face. She appears to be speaking or explaining something.']


In [10]:
video_input[0].shape

NameError: name 'video_input' is not defined

In [21]:
inputs = processor(
    videos = video_input,
    text = text_prompt,
    padding=True,
    return_tensors="pt",
)
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'pixel_values_videos', 'video_grid_thw', 'second_per_grid_ts'])

In [22]:
print(inputs['pixel_values_videos'].shape)

torch.Size([13728, 1176])


In [20]:
print(inputs['video_grid_thw'])

tensor([[24, 26, 22]])


In [11]:
trained_weights = torch.load("outputs\checkpoint-400\global_step400\mp_rank_00_model_states.pt")


RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory